In [ ]:
# =============================================================================
# RAG PIPELINE WITH LANGCHAIN
# Building a Retrieval-Augmented Generation System
# =============================================================================

# =============================================================================
# INSTALLATION (Run once per environment)
# =============================================================================

%pip install langchain langchain-community chromadb sentence-transformers \
             pypdf beautifulsoup4 requests python-dotenv langchain-ollama \
             matplotlib numpy


In [ ]:
# Core utilities
import os
import time
import json
from pathlib import Path
from datetime import datetime
from typing import List, Dict, Tuple
import numpy as np
import matplotlib.pyplot as plt

# Environment & document processing
from dotenv import load_dotenv
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, WebBaseLoader, TextLoader
from langchain_core.prompts import PromptTemplate


# Vector database & embeddings
import chromadb
from sentence_transformers import SentenceTransformer

# LLM interfaces
from langchain_ollama.llms import OllamaLLM           # standard LLM
from langchain_ollama.chat_models import ChatOllama  # chat interface
from langchain_ollama.embeddings import OllamaEmbeddings


# Web scraping
import requests
from bs4 import BeautifulSoup

print("✓ All imports successful")

In [ ]:
# =============================================================================
# 2. CONFIGURATION
# =============================================================================

# Load environment variables
load_dotenv()

# Pipeline configuration
CONFIG = {
    'chunk_size': 1000,
    'chunk_overlap': 200,
    'top_k_retrieval': 5,
    'llm_model': 'llama3.1',
    'collection_name': 'rag_documents',
    'embedding_model': 'all-MiniLM-L6-v2'
}

# Create project directories
Path("documents").mkdir(exist_ok=True)
Path("results").mkdir(exist_ok=True)

print("✓ Configuration loaded")
print(f"   LLM Model: {CONFIG['llm_model']}")
print(f"   Chunk size: {CONFIG['chunk_size']}, Overlap: {CONFIG['chunk_overlap']}")

In [ ]:
# =============================================================================
# 4. RAG PIPELINE OVERVIEW
# =============================================================================
# A. Document loading & chunking
# B. Vector database (ChromaDB)
# C. Retrieval-Augmented Generation (RAG)
# D. Evaluation & benchmarking
# E. Metrics & result analysis


In [ ]:
# LEILA 
#=============================================================================
# A. DOCUMENT LOADING & PREPROCESSING
# =============================================================================

class DocumentProcessor:
    """Handles loading and chunking documents from multiple sources."""
    
    def __init__(self, chunk_size: int = 1000, chunk_overlap: int = 200):
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            separators=["\n\n", "\n", ". ", " ", ""]
        )
        self.documents = []
        self.chunks = []
        print(f"✓ DocumentProcessor initialized (chunk_size={chunk_size}, overlap={chunk_overlap})")
    
    def load_pdf(self, pdf_path: str):
        """Load and process PDF document."""
        # TODO: Implement PDF loading
        # 1. Use PyPDFLoader(pdf_path)
        # 2. Load documents
        # 3. Add to self.documents with metadata (source, type='pdf')
        # Example:
        # loader = PyPDFLoader(pdf_path)
        # docs = loader.load()
        # self.documents.extend(docs)
        pass
    
    def load_text(self, text_path: str):
        """Load plain text file."""
        # TODO: Implement text file loading
        # 1. Use TextLoader(text_path)
        # 2. Load documents
        # 3. Add to self.documents with metadata
        pass
    
    def load_web(self, url: str):
        """Scrape and load web content."""
        # TODO: Implement web scraping
        # 1. Use WebBaseLoader([url])
        # 2. Load content
        # 3. Add to self.documents with metadata (source=url, type='web')
        pass
    
    def chunk_documents(self):
        """Split all loaded documents into chunks."""
        # TODO: Implement document chunking
        # 1. Use self.text_splitter.split_documents(self.documents)
        # 2. Store chunks in self.chunks
        # 3. Ensure each chunk has metadata: source, chunk_id, page_number (if available)
        pass
    
    def get_chunks_with_metadata(self) -> Tuple[List[str], List[Dict]]:
        """Return separate lists of chunk texts and metadata."""
        # TODO: Extract and return chunks and metadata
        # Return: ([chunk_text1, chunk_text2, ...], [metadata1, metadata2, ...])
        pass
    
    def print_stats(self):
        """Display document processing statistics."""
        print(f"\n📊 Document Processing Stats:")
        print(f"   Total documents loaded: {len(self.documents)}")
        print(f"   Total chunks created: {len(self.chunks)}")
        if self.chunks:
            avg_len = sum(len(str(c.page_content)) for c in self.chunks) / len(self.chunks)
            print(f"   Average chunk length: {avg_len:.0f} characters")


# Initialize document processor
doc_processor = DocumentProcessor(
    chunk_size=CONFIG['chunk_size'],
    chunk_overlap=CONFIG['chunk_overlap']
)

In [ ]:
# =============================================================================
# DOCUMENT LOADING SECTION
# =============================================================================
print("\n" + "="*60)
print("LOADING DOCUMENTS")
print("="*60)

# TODO: Load your documents here
# Example 1: Load PDFs
pdf_files = [
    "documents/paper1.pdf",
    "documents/paper2.pdf",
    "documents/paper3.pdf"
]
# for pdf in pdf_files:
#     if Path(pdf).exists():
#         doc_processor.load_pdf(pdf)
#         print(f"✓ Loaded: {pdf}")

# Example 2: Load web articles
# urls = [
#     "https://example.com/article1",
#     "https://example.com/article2"
# ]
# for url in urls:
#     doc_processor.load_web(url)
#     print(f"✓ Loaded: {url}")

# Example 3: Load text files
# doc_processor.load_text("documents/textbook.txt")

print(f"\n✓ Total documents loaded: {len(doc_processor.documents)}")

# Create chunks
doc_processor.chunk_documents()
doc_processor.print_stats()

# Preview first chunk
if doc_processor.chunks:
    print("\n📄 First chunk preview:")
    print(str(doc_processor.chunks[0].page_content)[:300] + "...")


In [ ]:
# MARIAM
#=============================================================================
# B. VECTOR DATABASE (CHROMADB)
# =============================================================================
import chromadb
from sentence_transformers import SentenceTransformer
from typing import List, Dict

CONFIG = {'embedding_model': 'all-MiniLM-L6-v2', 'collection_name': 'test'}
class VectorDB:
    """Manages vector embeddings and semantic search."""
    
    def __init__(self, collection_name: str = "rag_documents"):
        self.client = chromadb.Client()
        
        # Clear existing collection for fresh start
        try:
            self.client.delete_collection(name=collection_name)
        except:
            pass
        
        self.collection = self.client.create_collection(
            name=collection_name,
            metadata={"hnsw:space": "cosine"}
        )
        self.model = SentenceTransformer(CONFIG['embedding_model'])
        print(f"✓ VectorDB initialized (collection: {collection_name})")
    
    def add_documents(self, chunks: List[str], metadata: List[Dict] = None):
        """Add documents to vector database."""

        if not chunks:
            print("no chunks provided")
            return{'success':False, 'count':0}

        if metadata is None:
            metadata = [{'source': 'unknown', 'chunk_id': i} for i in range(len(chunks))]
    
        embeddings= self.model.encode(chunks)
        ids=[f"chunks{i}" for i in range(len(chunks))]

        self.collection.add(
             ids=ids,
             embeddings=embeddings.tolist(),
             documents=chunks,
             metadatas=metadata
        )
        
        return {'success':True, 'count':len(chunks)}
    
    def search(self, query: str, top_k: int = 5) -> List[Dict]:
        """Search for similar documents."""
        
        if self.collection.count()==0:
            print("⚠️  Database is empty. Add documents first.")
            return []
        query_embeddings=self.model.encode([query])[0]
        
        results=self.collection.query(
            query_embeddings=[query_embeddings.tolist()],
            n_results=top_k
        )
        
        formatted_results=[]
        for i in range(len(results['ids'][0])):
            formatted_results.append({
                'chunk': results['documents'][0][i],      # Original text
                'score': 1 - results['distances'][0][i],  # Convert distance to similarity
                'metadata': results['metadatas'][0][i],   # Source info
                'id': results['ids'][0][i]                # Chunk ID
            })
        
        return formatted_results
    
    def get_stats(self) -> Dict:
        """Return database statistics."""
        return {
            'total_chunks': self.collection.count(),
            'embedding_model': CONFIG['embedding_model'],
            'embedding_dimensions': self.model.get_sentence_embedding_dimension(),
            'database': 'ChromaDB',
            'similarity_metric':'cosine'
        }


    def clear_database(self):
        """Delete all documents from the collection."""
        collection_name = self.collection.name
        self.client.delete_collection(name=collection_name)
        self.collection = self.client.create_collection(
            name=collection_name,
            metadata={"hnsw:space": "cosine"}
        )
        print(f"✓ Database cleared")

# Initialize vector database
print("\n" + "="*60)
print("VECTOR DATABASE SETUP")
print("="*60)

vector_db = VectorDB(collection_name=CONFIG['collection_name'])

# TODO: Populate vector database
# chunks_text, chunks_metadata = doc_processor.get_chunks_with_metadata()
# vector_db.add_documents(chunks_text, chunks_metadata)

# Display stats
if result['success']:
    stats = vector_db.get_stats()
    print(f"\n📊 Vector Database Stats:")
    for key, value in stats.items():
        print(f"   {key}: {value}")

# Test search functionality
print("\n🔍 Testing vector search...")
# test_query = "What is retrieval-augmented generation?"
# results = vector_db.search(test_query, top_k=3)
# for i, result in enumerate(results, 1):
#     print(f"\nResult {i} (Score: {result['score']:.3f}):")
#     print(f"Source: {result['metadata'].get('source', 'unknown')}")
#     print(f"Text: {result['chunk'][:150]}...")


In [ ]:
# VERZHINIA 
#=============================================================================
# C. RAG PIPELINE CONSTRUCTION
# =============================================================================

# Define RAG prompt template
RAG_PROMPT_TEMPLATE = """You are a helpful AI assistant. Answer the question based ONLY on the provided context.

Context:
{context}

Question: {question}

Instructions:
- Answer using ONLY information from the context above
- If the context doesn't contain relevant information, say: "I don't have enough information to answer this question."
- Cite sources when possible
- Be concise and accurate

Answer:"""

prompt_template = PromptTemplate(
    template=RAG_PROMPT_TEMPLATE,
    input_variables=["context", "question"]
)


class RAGPipeline:
    """Complete RAG pipeline: retrieve → format → generate."""
    
    def __init__(self, vector_db: VectorDB, llm_model: str, top_k: int = 5):
        self.vector_db = vector_db
        self.top_k = top_k
        self.prompt_template = prompt_template
        
        # Initialize LLM (choose based on model name)
        if 'gpt' in llm_model:
            self.llm = ChatOpenAI(model=llm_model, temperature=0)
        else:
            # For Ollama local models
            from langchain_ollama import Ollama
            self.llm = Ollama(model=llm_model)
        
        print(f"✓ RAG Pipeline initialized with {llm_model}")
    
    def retrieve(self, query: str) -> List[Dict]:
        """Retrieve relevant chunks from vector database."""
        # TODO: Use self.vector_db.search() to get top_k results
        pass
    
    def format_context(self, retrieved_chunks: List[Dict]) -> str:
        """Format retrieved chunks into context string."""
        # TODO: Combine chunks into single context string
        # Include source information for citations
        # Example format:
        # Source 1 (document.pdf):
        # [chunk text]
        # 
        # Source 2 (article.txt):
        # [chunk text]
        pass
    
    def generate_response(self, query: str) -> Dict:
        """Complete RAG pipeline: retrieve + generate."""
        # TODO: Implement full RAG flow
        # 1. Retrieve relevant chunks
        # 2. Format context
        # 3. Create prompt using self.prompt_template
        # 4. Generate response with self.llm
        # 5. Return dict with: answer, sources, retrieved_chunks
        pass
    
    def query(self, question: str, return_sources: bool = True) -> Dict:
        """User-facing query method."""
        return self.generate_response(question)


# Initialize RAG pipeline
print("\n" + "="*60)
print("RAG PIPELINE INITIALIZATION")
print("="*60)

rag_pipeline = RAGPipeline(
    vector_db=vector_db,
    llm_model=CONFIG['llm_model'],
    top_k=CONFIG['top_k_retrieval']
)

# Test RAG pipeline
print("\n🧪 Testing RAG pipeline...")
# test_question = "What are the main components of a RAG system?"
# response = rag_pipeline.query(test_question)
# print(f"\n❓ Question: {test_question}")
# print(f"\n✅ Answer: {response['answer']}")
# print(f"\n📚 Sources: {', '.join(response['sources'])}")

In [ ]:
# HASSAN
#=============================================================================
# D. EVALUATION & TESTING
# =============================================================================

# Define test queries across different categories
TEST_QUERIES = {
    'factual_in_scope': [
        # TODO: Add questions that ARE answered in your documents
        "What is RAG?",
        "How does vector search work?",
    ],
    'synthesis': [
        # TODO: Add questions requiring multiple chunks
        "Compare different approaches to document chunking",
    ],
    'out_of_scope': [
        # TODO: Add questions NOT in your documents
        "What is the weather today?",
    ],
    'misleading': [
        # TODO: Add questions with false premises
        "Why is RAG less effective than keyword search?",
    ]
}

print("\n" + "="*60)
print("EVALUATION & TESTING")
print("="*60)
print(f"✓ Test dataset created with {sum(len(v) for v in TEST_QUERIES.values())} queries")


def run_rag_tests():
    """Test RAG pipeline across all query categories."""
    # TODO: Implement RAG testing
    # 1. Loop through TEST_QUERIES
    # 2. For each query, call rag_pipeline.query()
    # 3. Record: category, query, answer, sources, response_time
    # 4. Store results in list
    # 5. Return results
    pass


def run_base_llm_tests():
    """Test base LLM without RAG for comparison."""
    # TODO: Implement base LLM testing
    # 1. Initialize base LLM (without RAG)
    # 2. Test same queries
    # 3. Record: category, query, answer, response_time
    # 4. Return results
    pass


# Run tests
# rag_results = run_rag_tests()
# base_results = run_base_llm_tests()

In [ ]:
#RUMI
#=============================================================================
# E. EVALUATION METRICS (OPTIONAL)
# =============================================================================

class RAGEvaluator:
    """Evaluate RAG system performance."""
    
    def __init__(self):
        self.metrics = []
    
    def evaluate_faithfulness(self, answer: str, retrieved_chunks: List[str]) -> float:
        """Check if answer is grounded in retrieved chunks."""
        # TODO: Implement faithfulness scoring
        # Simple approach: check if answer phrases appear in chunks
        # Advanced: use LLM to verify claims
        # Return score 0-1
        pass
    
    def evaluate_relevance(self, query: str, answer: str) -> float:
        """Check if answer is relevant to query."""
        # TODO: Implement relevance scoring
        # Use embedding similarity between query and answer
        # Return score 0-1
        pass
    
    def detect_hallucination(self, answer: str, retrieved_chunks: List[str]) -> bool:
        """Detect if answer contains unsupported claims."""
        # TODO: Implement hallucination detection
        # Check for "I don't know" responses vs specific claims
        # Verify claims appear in chunks
        # Return True if hallucination detected
        pass
    
    def evaluate_response(self, query: str, answer: str, retrieved_chunks: List[str]) -> Dict:
        """Run all evaluation metrics."""
        # TODO: Call all evaluation methods
        # Return dict with all scores
        pass
    
    def aggregate_results(self) -> Dict:
        """Calculate aggregate metrics across all queries."""
        # TODO: Average all metrics
        # Return summary statistics
        pass


# evaluator = RAGEvaluator()
# TODO: Evaluate all RAG results and calculate aggregate metrics

In [ ]:
# =============================================================================
# RESULTS & VISUALIZATION
# =============================================================================

def save_results(rag_results, base_results, aggregate_metrics):
    """Save evaluation results to JSON file."""
    # TODO: Create results dictionary
    # Include: config, stats, results, metrics, timestamp
    # Save to results/rag_evaluation_{timestamp}.json
    pass


def visualize_results(rag_results, base_results):
    """Create visualizations comparing RAG vs base LLM."""
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('RAG Pipeline Evaluation Results', fontsize=16, fontweight='bold')
    
    # 1. Response Time Distribution
    if rag_results:
        response_times = [r.get('response_time', 0) for r in rag_results]
        axes[0, 0].hist(response_times, bins=20, color='skyblue', edgecolor='black')
        axes[0, 0].set_title('Response Time Distribution')
        axes[0, 0].set_xlabel('Time (seconds)')
        axes[0, 0].set_ylabel('Frequency')
        axes[0, 0].axvline(np.mean(response_times), color='red', 
                          linestyle='--', label=f'Mean: {np.mean(response_times):.2f}s')
        axes[0, 0].legend()

    
    # 2. Success Rate by Category
    if rag_results:
        categories = {}
        for result in rag_results:
            cat = result.get('category', 'unknown')
            categories[cat] = categories.get(cat, 0) + 1
        
        axes[0, 1].bar(categories.keys(), categories.values(), color='lightgreen', edgecolor='black')
        axes[0, 1].set_title('Queries by Category')
        axes[0, 1].set_xlabel('Category')
        axes[0, 1].set_ylabel('Count')
        axes[0, 1].tick_params(axis='x', rotation=45)

    # 3. RAG vs Base LLM Comparison
    if base_results:
        metrics = ['Faithfulness', 'Relevance']
        rag_scores = [0.8, 0.85]  # Example values
        base_scores = [0.5, 0.6]
        
        x = np.arange(len(metrics))
        width = 0.35
        
        axes[1, 0].bar(x - width/2, rag_scores, width, label='RAG', color='cornflowerblue')
        axes[1, 0].bar(x + width/2, base_scores, width, label='Base LLM', color='lightcoral')
        axes[1, 0].set_title('RAG vs Base LLM Performance')
        axes[1, 0].set_ylabel('Score')
        axes[1, 0].set_xticks(x)
        axes[1, 0].set_xticklabels(metrics)
        axes[1, 0].legend()
        axes[1, 0].set_ylim([0, 1])


# Save and visualize results
# save_results(rag_results, base_results, aggregate_metrics)
# visualize_results(rag_results, base_results)

print("\n" + "="*60)
print("FRAMEWORK COMPLETE")
print("="*60)
print("Next steps:")
print("1. Implement all TODO sections")
print("2. Load your documents (Section A)")
print("3. Test retrieval and generation")
print("4. Run evaluation tests")
print("5. Prepare presentation materials")

1. finish code(all) (rumi's part(hasan) & visualization(mariam)
2. documentation (all)
3. presentation (leila)
4. UI (verzhi)

online- meeting 15th Jan
on- campus meeting 19th Jan